# Inspecting and Modifying NZUpy Model Inputs

This notebook demonstrates the new input inspection and modification methods added in Phase 1 of the refactoring plan.

## 1. Set Up the NZUpy Model

First, let's import the necessary libraries and initialise the model.

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

# Add the project root to the path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Import the NZUpy class
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "input_inspection"
os.makedirs(output_dir, exist_ok=True)

# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## 2. Define Time Periods and Scenarios

Now we'll define the time periods and scenarios for our model run.

In [5]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Define multiple scenarios for demonstration
NZU.define_scenarios(['Baseline', 'High Forestry', 'Low Forestry'])

# Prime the model
NZU.prime()

Time periods defined:
  Optimisation: 2024-2050
Defined 3 scenarios: Baseline, High Forestry, Low Forestry
Model primed with 3 scenarios:
  [0] Baseline
  [1] High Forestry
  [2] Low Forestry


## 3. Configure Scenarios

We'll configure each scenario with different settings to demonstrate how to inspect and modify inputs.

In [6]:
# Configure scenarios with central configs as a starting point
NZU.use_central_configs(0)  # Baseline scenario
NZU.use_central_configs(1)  # High Forestry scenario
NZU.use_central_configs(2)  # Low Discount scenario

# Customise scenarios
# For High Forestry, use high forestry config
NZU.use_config(1, 'forestry', 'high')

# For Low Discount, use a custom discount rate
NZU.set_parameter('discount_rate', 0.03, component='stockpile', scenario_index=2)

Using central config for emissions in scenario 0 (Baseline)
Using central config for auction in scenario 0 (Baseline)
Using central config for industrial in scenario 0 (Baseline)
Using central config for forestry in scenario 0 (Baseline)
Using central config for demand_model in scenario 0 (Baseline)
Using central config for stockpile in scenario 0 (Baseline)
Using central configs for all components in model scenario 0 (Baseline)
Using central config for emissions in scenario 1 (High Forestry)
Using central config for auction in scenario 1 (High Forestry)
Using central config for industrial in scenario 1 (High Forestry)
Using central config for forestry in scenario 1 (High Forestry)
Using central config for demand_model in scenario 1 (High Forestry)
Using central config for stockpile in scenario 1 (High Forestry)
Using central configs for all components in model scenario 1 (High Forestry)
Using central config for emissions in scenario 2 (Low Forestry)
Using central config for auction in

## 4. Exploring Available Inputs

Let's explore what inputs are available for each component.

In [7]:
# List available configurations for different components
print("Available emissions configs:", NZU.data_handler.list_available_configs('emissions'))
print("Available auction configs:", NZU.data_handler.list_available_configs('auction'))
print("Available forestry configs:", NZU.data_handler.list_available_configs('forestry'))
print("Available demand model configs:", NZU.data_handler.list_available_configs('demand_model'))
print("Available stockpile configs:", NZU.data_handler.list_available_configs('stockpile'))

Available emissions configs: ['CCC_CPR', 'CCC_DP', 'CCC_mid', 'central']
Available auction configs: ['CCC_2024', 'central']
Available forestry configs: ['central', 'high', 'low']
Available demand model configs: ['95pc_lower', '95pc_upper', 'central', 'std_error', 'stde_lower', 'stde_upper']
Available stockpile configs: ['CCC22_central', 'CCC22_high', 'CCC22_low', 'CCC24_central', 'CCC24_high', 'CCC24_low', 'EY24_central', 'EY24_high', 'EY24_low', 'MFE24_central', 'MFE24_high', 'MFE24_low', 'central', 'high', 'low']


## 5. Using the New Inspection Methods

### 5.1 Showing All Inputs for a Component

In [8]:
# Show all stockpile inputs for Baseline scenario
NZU.show_inputs('stockpile', scenario_name='Baseline')

# Show all forestry inputs for High Forestry scenario
NZU.show_inputs('forestry', scenario_name='High Forestry')


=== Stockpile Configuration for Scenario 'Baseline' ===
Config name: central

Stockpile Parameters:
  Initial Stockpile: 159,902 kt CO₂-e
  Initial Surplus: 68,100 kt CO₂-e
  Liquidity Factor: 12.00%
  Discount Rate: 5.00%
  Payback Period: 25 years
  Stockpile Usage Start Year: 2024
  Stockpile Reference Year: 2023

=== Forestry Configuration for Scenario 'High Forestry' ===
Config name: high

Forestry Supply Volumes (first 5 years):
  2020: 3,546 kt CO₂-e
  2021: 3,542 kt CO₂-e
  2022: 3,599 kt CO₂-e
  2023: 3,802 kt CO₂-e
  2024: 4,247 kt CO₂-e
  ... 26 more years

Forestry Variables Available:
  - ('central', 'forestry_held')
  - ('low', 'forestry_held')
  - ('high', 'forestry_held')
  - ('central', 'forestry_surrender')
  - ('low', 'forestry_surrender')
  - ('high', 'forestry_surrender')


{'supply': Config       central          high           low  forestry_supply
 year                                                             
 2020     3546.167808   3546.167808   3546.167808      3546.167808
 2021     3542.064355   3542.064355   3542.064355      3542.064355
 2022     3598.566476   3598.566476   3598.566476      3598.566476
 2023     3802.135656   3803.004512   2865.638823      3802.135656
 2024     4246.851987   4242.859705   3293.547562      4246.851987
 2025     5655.987954   5651.088284   4687.032988      5655.987954
 2026     7820.489496   7811.517458   6834.809118      7820.489496
 2027    10373.843520  10361.808840   9349.204068     10373.843520
 2028    12659.601730  12652.535920  11539.405350     12659.601730
 2029    14212.435870  14230.949920  12930.702570     14212.435870
 2030    14942.364970  15059.816030  13392.501000     14942.364970
 2031    15405.035900  15730.199910  13516.113770     15405.035900
 2032    15800.122780  16434.238440  13539.520920   

### 5.2 Listing Available Parameters and Series

In [9]:
# List parameters available for stockpile component
NZU.list_parameters('stockpile', scenario_name='Baseline')

# List time series available for forestry component
NZU.list_series('forestry', scenario_name='High Forestry')


Available parameters for 'stockpile' in scenario 'Baseline':
  - initial_stockpile
  - initial_surplus
  - liquidity_factor
  - discount_rate
  - payback_period
  - stockpile_usage_start_year
  - stockpile_reference_year

Available time series for 'forestry' in scenario 'High Forestry':
  - central
  - high
  - low
  - forestry_supply
  - ('central', 'forestry_held')
  - ('low', 'forestry_held')
  - ('high', 'forestry_held')
  - ('central', 'forestry_surrender')
  - ('low', 'forestry_surrender')
  - ('high', 'forestry_surrender')


['central',
 'high',
 'low',
 'forestry_supply',
 ('central', 'forestry_held'),
 ('low', 'forestry_held'),
 ('high', 'forestry_held'),
 ('central', 'forestry_surrender'),
 ('low', 'forestry_surrender'),
 ('high', 'forestry_surrender')]

### 5.3 Showing Specific Parameters

In [10]:
# Show discount_rate for each scenario
NZU.show_parameter('discount_rate', 'stockpile', scenario_name='Baseline')
NZU.show_parameter('discount_rate', 'stockpile', scenario_name='High Forestry')
NZU.show_parameter('discount_rate', 'stockpile', scenario_name='Low Discount')

# Show initial stockpile balance
NZU.show_parameter('initial_stockpile', 'stockpile', scenario_name='Baseline')


=== Stockpile Configuration for Scenario 'Baseline' ===
Config name: central

Stockpile Parameters:
  Initial Stockpile: 159,902 kt CO₂-e
  Initial Surplus: 68,100 kt CO₂-e
  Liquidity Factor: 12.00%
  Discount Rate: 5.00%
  Payback Period: 25 years
  Stockpile Usage Start Year: 2024
  Stockpile Reference Year: 2023

Parameter 'discount_rate' for stockpile in scenario 'Baseline':
  Value: 5.00%

=== Stockpile Configuration for Scenario 'High Forestry' ===
Config name: central

Stockpile Parameters:
  Initial Stockpile: 159,902 kt CO₂-e
  Initial Surplus: 68,100 kt CO₂-e
  Liquidity Factor: 12.00%
  Discount Rate: 5.00%
  Payback Period: 25 years
  Stockpile Usage Start Year: 2024
  Stockpile Reference Year: 2023

Parameter 'discount_rate' for stockpile in scenario 'High Forestry':
  Value: 5.00%


ValueError: Unknown scenario name: 'Low Discount'. Available scenarios: Baseline, High Forestry, Low Forestry

### 5.4 Showing Time Series Data

In [ ]:
# Show forestry supply data for both scenarios
NZU.show_series('forestry_supply', 'forestry', scenario_name='Baseline')
NZU.show_series('forestry_supply', 'forestry', scenario_name='High Forestry')

# Show auction volumes
NZU.show_series('base_volume', 'auction', scenario_name='Baseline')

## 6. Modifying Inputs

### 6.1 Modifying Parameters

In [ ]:
# Change liquidity factor for Baseline scenario
NZU.set_parameter('liquidity_factor', 0.15, 'stockpile', scenario_name='Baseline')

# Verify the change
NZU.show_parameter('liquidity_factor', 'stockpile', scenario_name='Baseline')

# Change payback period for High Forestry scenario
NZU.set_parameter('payback_period', 20, 'stockpile', scenario_name='High Forestry')

# Verify the change
NZU.show_parameter('payback_period', 'stockpile', scenario_name='High Forestry')

### 6.2 Using set_series (Future Feature)

In [ ]:
# Note: set_series is not yet implemented in Phase 1
# This is just an example of how it will work in the future

# Get current auction volumes
#auction_volumes = NZU.show_series('base_volume', 'auction', scenario_name='Baseline')

# Modify auction volumes (increase by 10%)
#modified_volumes = auction_volumes * 1.1

# Set modified auction volumes
#NZU.set_series('base_volume', modified_volumes, 'auction', scenario_name='Baseline')

## 7. Running the Model and Analysing Results

After inspecting and modifying inputs, you can run the model and analyse the results.

In [ ]:
# Run the model with all scenarios
results = NZU.run()

# Access scenario results
baseline_results = NZU.get_scenario_result('Baseline')
high_forestry_results = NZU.get_scenario_result('High Forestry')
low_discount_results = NZU.get_scenario_result('Low Discount')

# Compare carbon prices across scenarios
print("\nFinal Carbon Prices:")
print(f"Baseline: ${baseline_results['final_price']:.2f}")
print(f"High Forestry: ${high_forestry_results['final_price']:.2f}")
print(f"Low Discount: ${low_discount_results['final_price']:.2f}")

## 8. Conclusion

These new inspection and modification methods make it much easier to:
- Understand what inputs are available for each component
- View current input values for a scenario
- Make targeted modifications to parameters

In future phases of the refactoring, we'll add:
- The ability to modify time series data (set_series)
- A unified DataFrame structure for all model data
- More advanced input validation and documentation